In [19]:
import networkx as nx
import pandas as pd

edges = pd.read_csv("dblp_edges.csv", delimiter=";")
G = nx.from_pandas_edgelist(edges)

G_ = nx.DiGraph(G.edges)
G_

In [1]:
# graph linearizations: input list of edges, output list of nodes? 
# what do we want when we progressively sample a graph? the network, i guess
# so we need a visualization of the graph 
#  - adjacency matrix? not really practical, would just show a matrix slowly evolving
#  - node-link diagram? new chunk --> new node layout?
#  - since we so-far have been using a scatter plot, it would make sense to visualize the graph 
#    along its nodes, and then gradually update the layout with every "new" node.
#  - this seems like its own can of worms ...

# if you linearize graph by node weights, okay, but then to render those nodes, you need their
# edges ... so output nodes+edges to all nodes that are currently there?
import pandas as pd
import numpy as np
import networkx as nx
from typing import List

# data is taken from the data mining course exercises, who took it from 
# http://nrvis.com/download/data/cit/cit-DBLP.zip
# and it only contains the edges of the graph
df = pd.read_csv("dblp_edges.csv", delimiter=";")


def random_linearization(edges: pd.DataFrame) -> List[int]: 
  '''Random shuffling of the nodes of the graph.'''
  # discover all nodes
  sources = edges["source"]
  targets = edges["target"]
  nodes = pd.concat([sources, targets]).unique()

  # shuffle that list
  random_index = np.random.rand(len(nodes))
  random_order = np.argsort(random_index)

  return list(nodes[random_order])

def connect_components(G: nx.Graph) -> nx.Graph:
    '''Adds simple edges between disconnected components of a given graph, thereby making it
       connected.'''
    components = nx.connected_components(G)
    last_node = None

    for component in components:
      if last_node:
        G.add_edge(last_node, list(component)[0])
      last_node = list(component)[0]

    return G

def basic_weighted_linearization(edges: pd.DataFrame) -> List[int]:
  '''An implementation of the naïve linearization algorithm without optimization as a basic
     proof-of-concept of using graph algorithms in the pipeline.'''
  G_ = nx.from_pandas_edgelist(edges)

  # graph may be disconnected, so as a first step, add edges between its conn. components
  G_ = connect_components(G_)

  # also, these self-looping edges (node, node) break the linearization algo, so remove them
  G_.remove_edges_from(nx.selfloop_edges(G_))
  G = nx.from_edgelist(G_.edges)

  latest_G = G
  nodes = list(G.nodes)

  # perform the linearization algorithm, locally ordering the graph by replacing left and right 
  # neighbors until all nodes have degree 2, except a start and an end with degree 1
  while((pd.DataFrame(latest_G.degree)[1] > 2).sum() > 0):
    new_G = nx.Graph()
    new_G.nodes = G.nodes

    for node in nodes:
      neighbors = np.array(list(latest_G.neighbors(node)))
      left_neighbors = neighbors[neighbors <= node]
      right_neighbors = neighbors[neighbors > node]

      # # check if node is already sorted into its neighborhood
      if len(left_neighbors) == 1 and len(right_neighbors) == 1:
        new_G.add_edges_from([[left_neighbors[0], node], [node, right_neighbors[0]]])
        continue

      # sort neighbors by their weight
      left_neighbors = left_neighbors[np.argsort(left_neighbors)]
      right_neighbors = right_neighbors[np.argsort(right_neighbors)]

      # left neighbors of n is an ordered list [u1, ..., ul]
      # transform this into edges [(u1, u2), (u2, u3), ..., (ul, n)]
      if len(left_neighbors) > 0:
        left_edges = np.empty((len(left_neighbors), 2)).astype(int)

        left_edges[:, 0] = left_neighbors
        left_edges[:len(left_neighbors) - 1, 1] = left_neighbors[1:]
        left_edges[len(left_neighbors) - 1, 1] = node

        new_G.add_edges_from(left_edges)

      # right neighbors of n is an ordered list [w1, ..., wm]
      # transform this into edges [(n, w1), ..., (wm-1, wm)]
      if len(right_neighbors) > 0:
        right_edges = np.empty((len(right_neighbors), 2)).astype(int)

        right_edges[1:, 0] = right_neighbors[0:len(right_neighbors) - 1]
        right_edges[:, 1] = right_neighbors
        right_edges[0, 0] = node

        new_G.add_edges_from(right_edges)

    latest_G = new_G

  # start and endpoint are the two nodes with degree 1
  degrees = np.array(latest_G.degree)
  endpoints = list(degrees[degrees[:, 1] == 1][:, 0])

  return list(nx.shortest_path(latest_G, endpoints[0], endpoints[1]))

def basic_linearization(edges: pd.DataFrame) -> list[int]:
  G_ = nx.from_pandas_edgelist(edges)

  # graph may be disconnected, so as a first step, add edges between its conn. components
  G_ = connect_components(G_)
  # also, these self-looping edges (node, node) break the linearization algo, so remove them
  G_.remove_edges_from(nx.selfloop_edges(G_))

  # furthermore, all cycles break this algo, so remove them
  is_cyclic = True
  while is_cyclic:
    try: 
      cycle = nx.find_cycle(G_)
      G_.remove_edge(*cycle[0])
    except:
      is_cyclic = False

  print("done removing cycles")

  # cycles = nx.cycles(G_)
  # G_ = nx.dag
  # for cycle in cycles:
  #   G_.more
  G = nx.DiGraph(G_.edges)

  nodes = list(G.nodes)

  # perform the linearization algorithm, locally ordering the graph by replacing left and right 
  # neighbors until all nodes have degree 2, except a start and an end with degree 1
  while((pd.DataFrame(G.degree)[1] > 2).sum() > 0):
    print((pd.DataFrame(G.degree)[1] > 2).sum())
    for node in nodes:
      left_neighbors = list(G.predecessors(node))
      in_edges = list(G.in_edges(node))

      if len(left_neighbors) > 1:
        G.remove_edges_from(in_edges)
        left_edges = np.empty((len(left_neighbors), 2)).astype(int)

        left_edges[:, 0] = left_neighbors
        left_edges[:len(left_neighbors) - 1, 1] = left_neighbors[1:]
        left_edges[len(left_neighbors) - 1, 1] = node

        G.add_edges_from(left_edges)

    for node in nodes:
      right_neighbors = list(G.successors(node))
      out_edges = list(G.out_edges(node))

      if len(right_neighbors) > 1:
        G.remove_edges_from(out_edges)
        right_edges = np.empty((len(right_neighbors), 2)).astype(int)

        right_edges[1:, 0] = right_neighbors[0:len(right_neighbors) - 1]
        right_edges[:, 1] = right_neighbors
        right_edges[0, 0] = node

        G.add_edges_from(right_edges)

  # start and endpoint are the two nodes with degree 1
  degrees = np.array(G.degree)
  endpoints = list(degrees[degrees[:, 1] == 1][:, 0])

  return list(nx.shortest_path(G, endpoints[0], endpoints[1]))

def shortest_path_linearization(edges: pd.DataFrame) -> List[int]:
  '''Linearize the graph along its shortest path, for cases where there are no weights assigned to 
     the nodes.'''
  pass

def spanning_tree_linearization(edges: pd.DataFrame) -> List[int]:
  '''Linearize the graph by traversing one of its spanning trees via depth-first search.'''
  G = nx.from_pandas_edgelist(edges)

  # graph may be disconnected, so as a first step, add edges between its conn. components
  G = connect_components(G)

  # I guess you could start by computing the spanning tree?
  spanning_tree = nx.minimum_spanning_tree(G)

  # then use dfs through that tree
  dfs_tree = nx.dfs_tree(spanning_tree)
  root = list(dfs_tree.nodes)[0]
  return list(nx.dfs_preorder_nodes(dfs_tree, source=root))
  

def edgeless_linearization(edges: pd.DataFrame) -> List[int]:
  '''Sorts the nodes only based on their value (i.e., not using graph algorithms at all).'''
  G = nx.from_pandas_edgelist(edges)
  nodes = np.array(G.nodes)
  
  return list(nodes[nodes.argsort()])

In [2]:
import pandas as pd
import numpy as np
import networkx as nx

# data is taken from the data mining course exercises, who took it from 
# http://nrvis.com/download/data/cit/cit-DBLP.zip
# and it only contains the edges of the graph
edges = pd.read_csv("dblp_edges.csv", delimiter=";")

# edges = pd.DataFrame([
#   (1, 1),
#   (2, 3),
#   (3, 1)
# ], columns=["source", "target"])

# G = nx.from_pandas_edgelist(edges, create_using=nx.DiGraph)

# basic_weighted_linearization(edges)
basic_linearization(edges)



done removing cycles
1555
598
493
359
293
253
221
202
173
155
145
139
136
128
117
111
107
104
97
92
91
86
78
78
77
77
76
74
73
74
71
70
69
67
65
63
63
62
61
59
58
59
59
57
56
53
51
50
49
48
48
48
48
47
46
45
45
45
45
45
45
45
45
45
45
44
44
44
43
42
42
42
42
40
38
38
39
39
38
37
37
37
37
36
36
36
34
34
34
34
34
34
34
33
33
33
32
32
32
32
32
31
31
30
30
29
30
30
30
30
30
30
29
30
30
29
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
26
26
26
26
26
26
26
25
25
25
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
23
22
22
22
22
22
22
22
21
21
21
21
21
21
21
21
21
21
21
21
21
20
20
20
20
20
19
19
19
19
19
19
19
19
19
19
19
19
19
18
18
18
17
17
17
17
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
15
15
15
15
15
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13


[5906,
 348,
 10523,
 309,
 12697,
 158,
 2049,
 3488,
 349,
 203,
 10971,
 3358,
 350,
 11358,
 582,
 4090,
 648,
 12287,
 98,
 6743,
 80,
 15699,
 946,
 11307,
 270,
 1833,
 1351,
 17338,
 52,
 675,
 1110,
 993,
 5728,
 8531,
 1332,
 1983,
 180,
 12194,
 1371,
 1607,
 464,
 1203,
 1579,
 11038,
 5814,
 9848,
 9566,
 2660,
 12898,
 9979,
 1362,
 2111,
 1087,
 10019,
 999,
 8609,
 5816,
 10035,
 9594,
 1934,
 11458,
 10885,
 9581,
 8688,
 5694,
 11781,
 138,
 16820,
 1658,
 12735,
 9404,
 2748,
 105,
 12762,
 1433,
 3090,
 131,
 12771,
 2282,
 3175,
 399,
 39,
 17629,
 1980,
 9623,
 12880,
 3930,
 3766,
 346,
 9145,
 1510,
 2456,
 614,
 13335,
 53,
 1963,
 463,
 11996,
 4072,
 3789,
 10738,
 13346,
 3096,
 4242,
 5822,
 9176,
 12423,
 2437,
 8949,
 9740,
 17506,
 151,
 480,
 5858,
 4272,
 10844,
 652,
 2164,
 12414,
 865,
 1365,
 1821,
 237,
 2666,
 187,
 9548,
 626,
 5791,
 11127,
 2280,
 11066,
 10777,
 11255,
 249,
 12304,
 11172,
 5867,
 2873,
 273,
 10962,
 10776,
 9255,
 2614,
 1